### 5 mrt
Uitgezocht welke tool we moeten gebruiken om quality controle uit te voeren op de minion data. Fastqc schijnt niet goed te werken op minion data, dus we gaan Nanoplot gebruiken. deze is te installeren via conda: `conda install -c bioconda nanoplot`

!To do:
- data verkennen
- inlezen nanoplot

### 11 mrt
Martijn heeft uitgelegd wat de bedoeling nu is. We gaan alle drie een andere tool (ik zoek kraken2 uit) gebruiken om het bicrobioom te bepalen en de resultaten daarvan vervolgens met elkaar vergelijken.

## 12 mrt
kraen2 installeren in conda: `conda install -c bioconda kraken2`
database uitzoeken: \
16S_Greengenes: [Archaea and Bacteria](https://pmc.ncbi.nlm.nih.gov/articles/PMC1489311/) \
16S silvia: [Bacteria, Archaea and Eukarya](https://www.arb-silva.de/)

16S lijkt uitgebreider, dus deze gaan we gebruiken.

commando: \
kraken2 : start kraken2 \
--db /data/datasets/16S_Silva : de database die we gebruiken \
--threads 8 : threads \
--report kraken2_report.txt : samenvatting rapport \
--output kraken2_output.txt : de output per read \
/commons/Themas/Thema07/metagenomics/zoutstress/16S/.. : input (seq data)

de input moet via een loop ingevuld worden: \
de mappen zijn: \
zoet_20_30cm_rep1  zoet_20_30cm_rep2  zout_20_30cm_rep1  zout_20_30cm_rep2 \
met daarin: \ 
FAY08769_pass_barcode07_85e26567_1cef8301_N.fastq.gz (waarin N een int is)

Maar omdat ze overlappingen hebben moeten ze in aparte mappen blijven.

Dus per map uitvoeren. acc_list.txt maken daarvoor.